In [4]:
#recognize words: not "-', don't make difference between some similar words
#maybe use third parties vectorizer :)
#after exlusion of weird words ^^
#requires build a list of only words first


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.stem import WordNetLemmatizer
%matplotlib inline

In [6]:
data_df = pd.read_csv("data/training/data_train.csv", dtype='unicode')
data_df.fillna(value='', inplace=True)
data_df = data_df.apply(lambda x: x.astype(str).str.lower())
data_df[['clarity','conciseness']] = data_df[['clarity','conciseness']].apply(pd.to_numeric)
train_df, test_df = train_test_split(data_df, test_size = 0.2)

NameError: name 'train_test_split' is not defined

In [ ]:
def getListCategories(train_df):
    listCat=[]
    for i in range(1,len (train_df)):
        category = str(train_df['category_lvl_1'][i])+"/"+str(train_df['category_lvl_2'][i])+"/"+str(train_df['category_lvl_3'][i])
        if category not in listCat:
            listCat.append(category)
        
    print("number of categories")
    print(len(listCat))
    return listCat
    

In [ ]:
def special_match(strg, search=re.compile(r'[^a-zA-Z0-9.]').search):
    return not bool(search(strg))

def is_code(word):
    if (not word.isalpha()) & (not word.isdigit()) & (special_match(word)) &(not is_measure(word)) :
        return True
    
    return False

def compressSeq(token):
    string=''
    for i in range(len(token)):
        if token[i].isdigit():
            string+='0'
        elif token[i].isalpha():
            string+='a'                        
        elif token[i] in ["?",".", ";", ",", "!",":"]:
            string+='.'
        else:
            string+='?'
        
    string = string.replace("0''", "0aa" )
    i=0
    while (i<len(string)-1):
        if string[i] == string[i+1]:
            string = string.replace(string[i]+string[i+1], string[i])
        else:
            i+=1                             
    return string

def is_measure(word):
    if (not word.isalpha()) & (not word.isdigit()):
        if compressSeq(word) == "0a":
            return True
    return False

def preProc(string, lem):
    string = string.replace('-','')
    
    words = string.split(" ")
    string2=''
    for word in words:
        if word.isalpha():
            word = word.lower()
            string2 += lem.lemmatize(word)+" "
            
    return string2


In [ ]:
def getListVocabulary(data, lem, lenMin = 2, maxSize =5000):           
    dictionary = []
    frequency = []
    
    
    
    
    #1 collect all words and number of appear
    for text in data:
        for word in  preProc(text, lem).split(" "):
            if (len(word) >= lenMin)  :
                if word not in dictionary:
                    dictionary.append(word)
                    frequency.append(1)
                else:
                    index=0
                    while dictionary[index] != word:
                        index+=1
                    frequency[index]+=1
                    
    #2 order words by their number of appear
                
    tempWord=''
    tempNum=0
    
    for i in range(len(dictionary)):
        for j in range(i, len(dictionary)):
            if frequency[i] < frequency[j]:
                tempWord = dictionary[i]
                dictionary[i] = dictionary[j]
                dictionary[j] = tempWord
                tempNum = frequency[i]
                frequency[i] = frequency[j]
                frequency[j] = tempNum
     
    #3 select sizeMax 
    newList=[]
    for i in range(maxSize):
        newList.append(dictionary[i])
            
    
    print("size of dictionnary")
    print(len(newList))
    print("number of words ignored:")
    print(len(dictionary) - len(newList))
    return newList

In [ ]:
lem = WordNetLemmatizer()
listVoc=getListVocabulary(train_df['title'], lem)
listCat=getListCategories(train_df)

In [ ]:
def embeddVector(title, category, listVoc, listCat, lem):    
    voc = preProc(title, lem).split(" ")
    vector=np.zeros((len(listVoc) +  len(listCat) + 3))
    
    for i in range(len(listVoc)):
        for word in voc:
            if word == listVoc[i]:
                vector[i] = 1
                
    for i in range(len(listCat)):
        if category == listCat[i]:
            vector[len(listVoc) + i] = 1
        
    words = title.split(" ")
    nbWords=len(words)
    nbCodes = 0
    nbMeasures=0
    for word in words:
        if is_code(word):
            nbCodes+=1
            
        
        if is_measure(word):
            nbMeasures+=1
            
    vector[len(listVoc) +  len(listCat) ] = nbWords
    vector[len(listVoc) +  len(listCat) +1] = nbCodes
    vector[len(listVoc) +  len(listCat) +2] = nbMeasures
    
    return vector    

In [ ]:
def vectorizeLabelledData(train_df):

    print("embedding  data")
    train_vect=[]
    label_clar_vect=[]
    label_conc_vect=[]

    for i in range(len(train_df)):
    #for i in range(100):

        title = train_df['title'][i]
        category = str(train_df['category_lvl_1'][i])+"/"+ str(train_df['category_lvl_2'][i])+"/"+ str(train_df['category_lvl_3'][i])  
        train_vect.append(embeddVector(title, category, listVoc, listCat, lem))
    
        label_clar_vect.append(train_df['clarity'][i])
        label_conc_vect.append(train_df['conciseness'][i])
    
        if i % (len(train_df)/100 ) == 0 :
            print( str(float(i) / float(len(train_df))))

    print('finished: '+str(i+1)+' samples')
    train_vect = np.matrix(train_vect)
    label_clar_vect = np.asarray(label_clar_vect)
    label_conc_vect = np.asarray(label_conc_vect)

    return(train_vect, label_clar_vect, label_conc_vect)

In [ ]:
(train_vect, label_clar_vect, label_conc_vect) = vectorizeLabelledData(train_df)

In [169]:
print ("Training the random forest...")
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0, nb_estimators = 100)
clf.fit(train_vect,label_clar_vect)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(train_vect,label_clar_vect)